In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import  DataLoader, Dataset
from torchvision import datasets, transforms


In [22]:
from PIL import Image

img = Image.open('chest_xray/test/NORMAL/IM-0007-0001.jpeg')
img.size

(2053, 1818)

### Transform is tool that used to modify the that we have given to the model

In [ ]:
transform = transforms.Compose(
    [
        transforms.Resize((2053, 1818)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5),(0.5,0.5))
    ]
)

In [ ]:
train_dataset = datasets.ImageFolder(root='chest_xray/train', transform=transform)
test_dataset = datasets.ImageFolder(root='chest_xray/test', transform=transform)
val_dataset = datasets.ImageFolder(root='chest_xray/val', transform=transform)

train_loader = DataLoader(dataset=train_dataset,batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=32, shuffle=False)
val_loader = DataLoader(dataset=val_dataset,batch_size=32, shuffle=False)

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes=2):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(2, 32, kernel_size=3, padding=1),  # input: 2 channels
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # reduce size

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        # flatten -> depends on input size (90x140)
        self.fc_layers = nn.Sequential(
            nn.Linear(64 * 22 * 35, 128),  # 90/2/2=22, 140/2/2=35
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # flatten
        x = self.fc_layers(x)
        return x


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN(num_classes=10).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)


In [13]:
losses_list = []

In [14]:
for epoch in range(10):  # number of epochs
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    losses_list.append(running_loss/len(train_loader))
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")


Epoch 1, Loss: 1.2880
Epoch 2, Loss: 0.6123
Epoch 3, Loss: 0.4509
Epoch 4, Loss: 0.3473
Epoch 5, Loss: 0.2901
Epoch 6, Loss: 0.2940
Epoch 7, Loss: 0.2477
Epoch 8, Loss: 0.2118
Epoch 9, Loss: 0.1934
Epoch 10, Loss: 0.1762


In [15]:
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total:.2f}%")


Validation Accuracy: 92.69%


In [16]:
correct, total = 0, 0
model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 92.46%
